In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import random

In [2]:
# Arguments
device = 'cuda'
epochs = 30

In [3]:
# 設置seed
seed = 198964
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
# Import data
train_data = datasets.MNIST('./MNIST_data', train=True, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                            ]))
test_data = datasets.MNIST('./MNIST_data', train=False, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,),(0.3081,))
                           ]))
val_data, test_data = torch.utils.data.random_split(dataset=test_data, lengths=[5000,5000])

In [5]:
# Dataloader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=3)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=1000, shuffle=True, num_workers=3)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=True, num_workers=3)

增加兩行convolution layers，並增加neuron數量

In [10]:
# module
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(5,5),stride=1, padding=0)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(5,5),stride=1, padding=0)
        self.fc1 = nn.Linear(16*4*4, 120) # make neuron 2x wider
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = torch.flatten(out, start_dim=1) #flatten
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
    
model = Net().to(device) # build model

建model並設置優化器

In [50]:
#define optimizer/loss function
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)
# optimizer = optim.RMSprop(model.parameters(), lr=0.1, momentum=0, alpha=0.5) # Learning rate here will be replaced by next function. By the way, alpha is set to 0.99 defaultly, and I set it to 0.5 . I don't want any momentum as well.

設定learning rate的排程，使他在多個epoch之後逐漸降低

In [12]:
# RMSprop learning rate scheduling
def adjust_learning_rate(optimizer, epoch):
    if epoch < 5: # After 5 epochs, everage loss always decrease slowly, so we can use smaller learning rate to avoid too much change
        lr = 0.005  # 0.01 is too big so let's try 0.005
    elif epoch < 15:
        lr = 0.0005 
    else: 
        lr = 0.00005

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [51]:
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=epochs, verbose=True)

Adjusting learning rate of group 0 to 1.0000e-02.


In [48]:
#training function
def train(model, loss_func, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

In [52]:
#Testing function
def test(model, loss_func, optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(val_loader):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
            test_loss += loss_func(output, target).item()
            pred = output.argmax(dim=1) # get the index of the max log-probability
            correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#run and save model
for epoch in range(1, epochs + 1):
    train(model, Loss, optimizer, epoch)
    test(model, Loss, optimizer)
    lr_scheduler.step()

Train Epoch: 1 [44928/60000 (100%)]	Loss: 0.581821

Test set: Average loss: 0.5045, Accuracy: 4222/5000 (84%)

Adjusting learning rate of group 0 to 9.9726e-03.
Train Epoch: 2 [44928/60000 (100%)]	Loss: 0.312030

Test set: Average loss: 0.2866, Accuracy: 4516/5000 (90%)

Adjusting learning rate of group 0 to 9.8907e-03.
Train Epoch: 3 [44928/60000 (100%)]	Loss: 0.215037

Test set: Average loss: 0.1941, Accuracy: 4701/5000 (94%)

Adjusting learning rate of group 0 to 9.7553e-03.
Train Epoch: 4 [44928/60000 (100%)]	Loss: 0.305573

Test set: Average loss: 0.1371, Accuracy: 4788/5000 (96%)

Adjusting learning rate of group 0 to 9.5677e-03.
Train Epoch: 5 [44928/60000 (100%)]	Loss: 0.091997

Test set: Average loss: 0.1064, Accuracy: 4835/5000 (97%)

Adjusting learning rate of group 0 to 9.3301e-03.
Train Epoch: 6 [44928/60000 (100%)]	Loss: 0.109248

Test set: Average loss: 0.0937, Accuracy: 4863/5000 (97%)

Adjusting learning rate of group 0 to 9.0451e-03.
Train Epoch: 7 [44928/60000 (100%)